In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
#pd.options.mode.copy_on_write = True
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
import multiprocessing as mp
from tqdm import tqdm
cosmo = FlatLambdaCDM(name='Planck18', H0=67.66, Om0=0.30966, Tcmb0=2.7255, Neff=3.046, m_nu=[0.  , 0.  , 0.06]* u.eV, Ob0=0.04897)
cores = 8

In [2]:
import base64
import gzip
import io
import logging
import os
import glob
import time
from copy import deepcopy
from datetime import datetime
from typing import Mapping, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.time import Time
from astropy.visualization import (
    AsymmetricPercentileInterval,
    ImageNormalize,
    LinearStretch,
    LogStretch,
)
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from colorama import init as colorama_init
from colorama import Fore, Style

#import threading
#from threading import Thread
import multiprocessing as mp

In [3]:
import GetLightcurve
import SendToSkyportal

In [4]:
crossmatch_df = pd.read_csv('../CrossMatch/DESIRT_one_arcsecond_crossmatch_final_list.csv')
DESIRT_lc = pd.read_csv('../LightCurves/DESIRT_lightcurves_final_list.csv')

In [5]:
crossmatch_df.head()

,STANDARD_NAME,21A,22A,22B,23A,23B,ra_obj,dec_obj
0,C202210020000005m002704,NaN,NaN,C202210020000005m002704,NaN,NaN,0.001904,-0.451054
1,T202210020000016p003460,NaN,NaN,T202210020000016p003460,NaN,NaN,0.006834,0.583234
2,T202210020000064p005207,NaN,NaN,T202210020000064p005207,NaN,NaN,0.026743,0.868546
3,T202308030000269p002005,NaN,NaN,NaN,NaN,T202308030000269p002005,0.112236,0.334703
4,T202210170000300p004846,NaN,NaN,T202210170000300p004846,NaN,NaN,0.125014,0.812758


In [7]:
GetLightcurve.full_dataframe([crossmatch_df["STANDARD_NAME"].iloc[0], crossmatch_df]).head()["LIM_MAG3"]

0    21.303990
1    21.216245
2    21.130671
3    22.737907
4    23.367237
Name: LIM_MAG3, dtype: float64

In [8]:
GetLightcurve.full_dataframe([crossmatch_df["STANDARD_NAME"].iloc[0], crossmatch_df]).columns

Index(['INDEX', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT', 'SEMESTER_NAME', 'STANDARD_NAME'],
      dtype='object')

In [9]:
base_name = "DESIRT_PIPE"
group_ids = [1721]
filter_ids = [1157]
instrument_id = 54
stream_id = 1006
token_skyportal = os.getenv("FRITZ_TOKEN")
endpoint = "fritz.science"
origin = "DESIRT_BOT"
filter_dict = {b'g': "desg", b'i': "desi", b'r': "desr", b'z': "desz", b'u': "desu"}

In [10]:
def api(method, path, data):
    headers = {'Authorization': f'token {token_skyportal}'}
    response = requests.request(method, path, json=data, headers=headers)
    return response

In [11]:
def obj_post_candidate(obj):
    """
    Post a candidate on SkyPortal.
    Creates new candidate(s) (one per filter)
    """
    data = {
        "id": obj["STANDARD_NAME"],
        "ra": obj["ra_obj"],
        "dec": obj["dec_obj"],
        "group_ids": group_ids,
        "filter_ids": filter_ids,
        "passed_at": Time(datetime.utcnow()).isot,
        "origin": origin,
    }
    
    response = api("POST", f"https://{endpoint}/api/candidates", data)
    return response

In [12]:
def obj_post_source(obj):
    """
    Add a new source to SkyPortal
    """
    data = {
        "id": obj["STANDARD_NAME"],
        "ra": obj["ra_obj"],
        "dec": obj["dec_obj"],
        "group_ids": group_ids,
        "origin": origin,
    }
    
    response = api("POST", f"https://{endpoint}/api/sources", data)
    return response

In [92]:
def obj_put_photometry(obj, crossmatch, snr_thresh=3):
    """Send photometry to Fritz."""
    
    df_photometry = GetLightcurve.full_dataframe([obj["STANDARD_NAME"], crossmatch])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]
    
    df_photometry.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"], inplace = True)
    filters = [filter_dict[i] for i in df_photometry["FILTER"]]
    
    photometry = {
        "obj_id": obj["STANDARD_NAME"],
        "group_ids": group_ids,
        "instrument_id": instrument_id,
        "mjd": df_photometry["MJD_OBS"].tolist(),
        #"flux": df_photometry["flux"].tolist(),
        #"fluxerr": df_photometry["fluxerr"].tolist(),
        "mag": df_photometry["MAG_FPHOT"].tolist(),
        "magerr": df_photometry["MAGERR_FPHOT"].tolist(),
        "snr": df_photometry["SNR_FPHOT"].tolist(),
        #"zp": df_photometry["zp"].tolist(),
        "magsys": "ab",
        "filter": filters,
        "limiting_mag_nsigma": 3,
        "limiting_mag": df_photometry["LIM_MAG3"].tolist(),
        "ra": None,
        "dec": None,
    }
    
    try:
        if np.isnan(photometry["mag"][0]) == True or \
                np.isnan(photometry["magerr"][0]):
            photometry["mag"][0] = None
            photometry["magerr"][0] = None
        # add check for SNR
        elif photometry["snr"][0] is not None:
            if photometry["snr"][0] < snr_thresh:
                photometry["mag"][0] = None
                photometry["magerr"][0] = None
        else:
            if photometry["ra"] is None and photometry["dec"] is None:
                photometry["ra"] = df_photometry["ra"].tolist()
                photometry["dec"] = df_photometry["dec"].tolist()
        if photometry["mag"][0] is None:
            photometry['snr'][0] = None
    except IndexError:
        print("WARNING: empty photometry")
        return
    
    photometry.pop('snr', None)
    
    response = api("PUT", f"https://{endpoint}/api/photometry", photometry)
    return response

In [14]:
def post_source(source):
    obj, crossmatch = source
    responses = []
    response_source = obj_post_source(obj)
    responses.append(response_source)
    if response_source.status_code == 200:
        response_phot = obj_put_photometry(obj, crossmatch)
        responses.append(response_phot)
    return responses
        

In [15]:
queue = [[crossmatch_df.iloc[i], crossmatch_df] for i in range(crossmatch_df.shape[0])]
response_list = []
for i in tqdm(queue):
    response_list.append(post_source(i))
    time.sleep(0.1)

100%|██████████| 2339/2339 [44:05<00:00,  1.13s/it]  


In [95]:
range(crossmatch_df.shape[0])

range(0, 2339)

In [16]:
np.save("responses.npz", np.asarray(response_list))

In [17]:
response_list = np.load("responses.npz.npy", allow_pickle=True)

In [35]:
counter = 0
index = []
index_first = []
for i in response_list:
    if(i[0].status_code != 200):
        index_first.append(counter)
    if(i[0].status_code == 200):
        if(i[1].status_code != 200):
            index.append(counter)
    counter+=1

In [51]:
queue = [[crossmatch_df.iloc[i], crossmatch_df] for i in range(crossmatch_df.shape[0])]

In [93]:
response_list_new = []
for i in tqdm([queue[i] for i in index]):
    response_list_new.append(post_source(i))
    time.sleep(0.1)

100%|██████████| 120/120 [01:50<00:00,  1.09it/s]


In [94]:
response_list_new

[[<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Response [200]>],
 [<Response [200]>, <Resp

In [72]:
new_queue = [queue[i] for i in index]

In [87]:
problem_df = GetLightcurve.full_dataframe([new_queue[0][0]["STANDARD_NAME"], new_queue[0][1]])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]

In [91]:
problem_df.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"])

,STANDARD_NAME,SEMESTER_NAME,SCIENCE_NAME,TEMPLATE_NAME,MJD_OBS,FILTER,DETPOS,X_OBJ,Y_OBJ,X_FPHOT,...,APER_FPHOT,ZP_FPHOT,LIM_MAG3,LIM_MAG5,MAG_FPHOT,MAGERR_FPHOT,SNR_FPHOT,STATUS_FPHOT,REAL,LIMIT
0,A202302171046011m060342,A202302171046011m060342,b'c4d_230202_071825_xxx_g_v1',b'c4d_170302_041423_xxx_g_v1',59977.304456,b'g',b'N6',613.603241,3439.711213,613.609601,...,5.61,28.815354,24.003513,23.448891,23.651845,0.216262,5.020311,b'q',True,False
1,A202302171046011m060342,A202302171046011m060342,b'c4d_230202_071944_xxx_r_v1',b'c4d_170302_041226_xxx_r_v1',59977.305373,b'r',b'N6',611.511136,3430.062211,611.517494,...,5.92,29.338620,23.708681,23.154059,23.614646,0.232453,4.670612,b'q',True,False
2,A202302171046011m060342,A202302171046011m060342,b'c4d_230202_072836_xxx_g_v1',b'c4d_170304_032950_xxx_g_v1',59977.311528,b'g',b'N1',1312.508545,588.191345,1312.516968,...,7.36,29.373822,23.755945,23.201323,23.591971,0.229834,4.723848,b'q',True,False
4,A202302171046011m060342,A202302171046011m060342,b'c4d_230202_073133_xxx_z_v1',b'c4d_180209_062307_xxx_z_v1',59977.313585,b'z',b'N1',1294.990948,547.881286,1294.999436,...,5.38,29.412729,22.980375,22.425753,23.591061,0.446776,2.430076,b'm',False,True
5,A202302171046011m060342,A202302171046011m060342,b'c4d_230205_080118_xxx_z_v1',b'c4d_180209_062307_xxx_z_v1',59980.334240,b'z',b'N1',1294.990948,547.881286,1294.999436,...,4.92,29.409567,22.446016,21.891395,23.270012,0.675385,1.607528,b'm',False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,A202302171046011m060342,A202302171046011m060342,b'c4d_240128_073814_xxx_g_v1',b'c4d_170302_041423_xxx_g_v1',60337.318222,b'g',b'N6',613.603241,3439.711213,613.609601,...,4.99,28.812065,22.339839,21.785217,22.415810,0.366441,2.962825,b'm',False,True
86,A202302171046011m060342,A202302171046011m060342,b'c4d_240128_074642_xxx_z_v1',b'c4d_180209_062307_xxx_z_ls9',60337.324107,b'z',b'N1',1294.991500,547.879106,1294.999988,...,4.85,29.363754,22.060685,21.506063,23.499097,1.222291,0.888250,b'm',False,True
87,A202302171046011m060342,A202302171046011m060342,b'c4d_240131_080857_xxx_r_v1',b'c4d_170302_041226_xxx_r_v1',60340.339549,b'r',b'N6',611.511136,3430.062211,611.517494,...,8.14,29.394555,22.926736,22.372114,23.188084,0.379191,2.863200,b'm',False,True
88,A202302171046011m060342,A202302171046011m060342,b'c4d_240131_081648_xxx_r_v1',b'c4d_170328_023019_xxx_r_ls9',60340.345008,b'r',b'N1',1314.569900,552.424938,1314.578449,...,7.17,29.386040,22.964928,22.410306,25.892600,4.307185,0.252067,b'm',False,True


In [84]:
problem_df.iloc[3]

INDEX                                        4
SCIENCE_NAME     b'c4d_230202_072836_xxx_g_v1'
TEMPLATE_NAME    b'c4d_170304_032950_xxx_g_v1'
MJD_OBS                                59977.3
FILTER                                    b'g'
DETPOS                                   b'N1'
X_OBJ                                  1312.51
Y_OBJ                                  588.191
X_FPHOT                                1312.52
Y_FPHOT                                587.714
X_nSTAR                                1330.58
Y_nSTAR                                560.953
X_nGAL                                 1312.06
Y_nGAL                                 587.529
X_ALT                                      NaN
Y_ALT                                      NaN
MAG_ALT                                    NaN
MAGERR_ALT                                 NaN
SNR_ALT                                    NaN
CNNSCORE_ALT                               NaN
ZMIN_TEMP                             -23.6034
ZMAX_TEMP    

In [71]:
response_list_new[3][1].json()

{'status': 'error',
 'message': 'Traceback (most recent call last):\n  File "/skyportal/skyportal/handlers/api/photometry.py", line 1592, in put\n    ids, upload_id = insert_new_photometry_data(\n  File "/skyportal/skyportal/handlers/api/photometry.py", line 892, in insert_new_photometry_data\n    save_data_using_copy(\n  File "/skyportal/skyportal/handlers/api/photometry.py", line 218, in save_data_using_copy\n    cursor.copy_from(\npsycopg2.errors.UniqueViolation: duplicate key value violates unique constraint "deduplication_index"\nDETAIL:  Key (obj_id, instrument_id, origin, mjd, fluxerr, flux)=(A202302021048243m051915, 54, None, 59977.31152824, 0.2669706227728932, 1.4425048005168717) already exists.\nCONTEXT:  COPY photometry, line 2\n\n',
 'data': {},
 'version': '0.9.dev0+git20240910.755d7fb3+fritz.cf6e52b'}

In [7]:
len(GetLightcurve.full_dataframe(["T202210260347253m155435", crossmatch_df]))

78